# Vertex AI Gen AI Evaluation Service による ADK エージェントの評価

## 1. 概要と事前準備

このノートブックの目的は、ADK で作成したエージェントの回答品質およびツール呼び出しの正確性を定量的に評価することです。

## 2. テスト対象エージェントの構築

評価の対象として、人事ポリシー（HR Policy）に関する問い合わせに回答するモックエージェントを構築しています。

* **モックツール**: ドキュメントのリストアップ (`list_policy_documents`)、本文取得 (`get_document_text`)、監査ログの記録 (`log_policy_audit`) の3つの関数が定義されています。
* **ADK エージェント**: `LlmAgent` を使用し、Gemini モデルをベースに上記ツールを扱えるよう構成されています。
* **チャットクライアント**: ストリーミング出力やツール実行履歴（トラジェクトリ）をパースするための `ChatClient` クラスが実装されています。

## 3. 評価手法

このノートブックでは、大きく分けて2つの評価アプローチを実演しています。

### A. レスポンス評価（ルーブリックベース）

正解データ（Ground Truth）を参照せずに、モデルの出力そのものを評価する手法です。

* **GENERAL_QUALITY メトリック**: Vertex AI が評価の観点（ルーブリック）を自動生成し、回答が適切かどうかを評価します。
* **実例**: 「日本語で回答しているか」「専門用語を文脈に沿って説明しているか」といった観点で評価結果が出力されます。

### B. トラジェクトリ評価（軌跡評価）

エージェントが「どのツールをどの順番で呼び出したか」を、事前に用意した正解（リファレンス）と比較して評価します。

* **シングルメトリック評価**: `log_policy_audit` などの特定のツールが確実に使用されたかをチェックします。
* **詳細なトラジェクトリ指標**: 以下のスコアで精度を測定します。
  * `trajectory_exact_match`: 順序も含めて完全に一致しているか。
  * `trajectory_in_order_match`: 正解に含まれるツールが、正しい順序で（間に他が入っても可）実行されたか。
  * `trajectory_any_order_match`: 順序を問わず、必要なツールがすべて含まれているか。
  * `trajectory_precision` / `trajectory_recall`: ツール使用の正確性と網羅性。



## 4. 検証結果の分析

インストラクション（指示）を変更することで、評価スコアがどのように変化するかが示されています。

* **指示の変更**: 「2種類以上のドキュメントを参照せよ」や「読む前にログを記録せよ」といった制約を追加し、それがトラジェクトリにどう影響するかを検証しています。
* **ログ解析**: 期待される順序と異なる実行（例：監査ログを先に記録するなど）が発生した場合にそれぞれのスコアがどのように変化するかが確認できます。

## 事前準備

In [1]:
import asyncio
import base64
import json
import os
import pandas as pd
import uuid
import vertexai

from vertexai.agent_engines import AdkApp
from google.adk.agents import LlmAgent
from google.adk.apps import App
from google.adk.artifacts import InMemoryArtifactService
from google.adk.events import Event
from google.adk.planners import BuiltInPlanner
from google.adk.plugins.save_files_as_artifacts_plugin import SaveFilesAsArtifactsPlugin
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import load_artifacts
from google.genai.types import Part, Content, Blob, GenerateContentConfig, ThinkingConfig, HttpOptions
from IPython.display import Markdown, display

# エージェントの回答に対するルーブリックベースの評価
from vertexai import types
from vertexai.types import RubricMetric

# エージェントのアクションに対するトラジェクトリ評価
from vertexai.preview.evaluation import EvalTask
from vertexai.preview.evaluation.metrics import (
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
    TrajectorySingleToolUse,
)

### Vertex AI Experiment を使用するための設定

In [2]:
[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'
BUCKET_URI = f'gs://{PROJECT_ID}'
EXPERIMENT_NAME = 'evaluate-adk-agent'

vertexai.init(project=PROJECT_ID, location=LOCATION, experiment=EXPERIMENT_NAME)

### 補助関数

In [3]:
def get_id(length=8):
    return uuid.uuid4().hex[:length]

def format_output_as_markdown(output):
    markdown = "### AI Response\n" + output["response"] + "\n\n"
    if output["predicted_trajectory"]:
        markdown += "### Function Calls\n"
        for call in output["predicted_trajectory"]:
            markdown += f"- **Function**: `{call['tool_name']}`\n"
            markdown += "  - **Arguments**\n"
            for key, value in call["tool_input"].items():
                markdown += f"    - `{key}`: `{value}`\n"
    return markdown

## テスト対象とするエージェントの定義

### モックツール

In [4]:
POLICY_DB = {
    "hr": [
        {"doc_id": "hr_2026_dress", "title": "オフィスのドレスコード"},
    ],
    "benefits": [
        {"doc_id": "ben_401k_match", "title": "401k拠出金およびマッチング拠出"},
        {"doc_id": "ben_med_gold", "title": "健康保険ゴールドプランの概要"},
    ]
}

DOCUMENT_CONTENT = {
    "hr_2026_dress": "ビジネスカジュアルが標準です。金曜日はカジュアルな服装が認められます。",
    "ben_401k_match": "会社は従業員の拠出額の最初の4%に対して100%のマッチングを行います。",
    "ben_med_gold": "健康保険のゴールドプランは予防医療費を100%カバーし、個人の免責額は5万円です。",
}

def list_policy_documents(category: str) -> list[dict]:
    """'hr' または 'benefits' のカテゴリに対応するドキュメントIDを返します。"""
    category = category.lower()
    return POLICY_DB.get(category, [])

def get_document_text(doc_id: str) -> str:
    """特定の doc_id のテキストを取得します。"""
    return DOCUMENT_CONTENT.get(doc_id, "エラー: ドキュメントIDが見つかりません。")

def log_policy_audit(doc_id: str, reason: str) -> str:
    """
    ドキュメントへのアクセスを監査ログに記録します。
    reason は ['personal', 'customer', 'other'] の1つを選択。
    """
    return "監査ログの記録に成功しました。"

### モックツールを使用するエージェントのテンプレート

In [5]:
def crate_adkapp(instruction, temperature=0.0):
    root_agent = LlmAgent(
        name='hr_policy_agent',
        model='gemini-2.5-flash',
        instruction=instruction,
        generate_content_config=GenerateContentConfig(temperature=temperature),
        tools=[list_policy_documents, get_document_text, log_policy_audit]
    )

    return AdkApp(
        agent=root_agent,
        app_name='hr_policy_agent',
    )

### AdkApp をテストするためのチャットクライアント

In [6]:
class ChatClient:
    def __init__(self, app, user_id='default_user'):
        self._app = app
        self._user_id = user_id
        self._session_id = None


    def parse_adk_output_to_dictionary(self, events):
        result, trajectory = [], []
        for event in events:
            if not ('content' in event and 'parts' in event['content']):
                continue
            for part in event['content']['parts']:
                if 'function_call' in part:
                    info = {
                        'tool_name': part['function_call']['name'],
                        'tool_input': part['function_call']['args'],
                    }
                    trajectory.append(info)
            if event['content']['role'] == 'model':
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    result.append(response)

        return {'response': '\n'.join(result), 'predicted_trajectory': trajectory}


    async def async_stream_query(self, message, show=False):
        if not self._session_id:
            session = await self._app.async_create_session(
                user_id=self._user_id,
            )
            self._session_id = getattr(session, 'id', None) or session['id']

        events = []
        async for event in self._app.async_stream_query(
            user_id=self._user_id,
            session_id=self._session_id,
            message=message,
        ):
            events.append(event)
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response and show:
                    print(response)

        return self.parse_adk_output_to_dictionary(events)


    def agent_parsed_outcome_sync(self, prompt):
        result = asyncio.run(self.async_stream_query(prompt, show=False))
        return result

## 「正解データ」の準備

In [7]:
instruction = '''
あなたは人事アシスタントです。
1. list_policy_documents を使用してドキュメントIDを検索してください。
2. get_document_text を使用して内容を取得してください。
3. ドキュメントのテキストに基づいてユーザーの質問に答えてください。

markdownを使用せずに、最終回答のみを日本語で出力します。

**重要**
ドキュメントを読んだ後は、監査のために必ず log_policy_audit を使用してアクセスを記録してください。
'''

In [8]:
chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.0))
query1 = '健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。'
events1 = await chat_client.async_stream_query(query1)

display(Markdown(format_output_as_markdown(events1)))

### AI Response
健保のゴールドプランの個人の免責額は5万円です。

### Function Calls
- **Function**: `list_policy_documents`
  - **Arguments**
    - `category`: `benefits`
- **Function**: `get_document_text`
  - **Arguments**
    - `doc_id`: `ben_med_gold`
- **Function**: `log_policy_audit`
  - **Arguments**
    - `doc_id`: `ben_med_gold`
    - `reason`: `personal`


In [9]:
query2 = '金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。'
events2 = await chat_client.async_stream_query(query2)

display(Markdown(format_output_as_markdown(events2)))

### AI Response
金曜日はカジュアルな服装が認められていますが、お客様との打ち合わせがある場合は、ビジネスカジュアルが適切です。

### Function Calls
- **Function**: `list_policy_documents`
  - **Arguments**
    - `category`: `hr`
- **Function**: `get_document_text`
  - **Arguments**
    - `doc_id`: `hr_2026_dress`
- **Function**: `log_policy_audit`
  - **Arguments**
    - `reason`: `customer`
    - `doc_id`: `hr_2026_dress`


In [10]:
# レスポンス評価用のデータ
response_eval_data = {
    'prompt': [
        query1,
        query2,
    ],
    'response': [
        events1['response'],
        events2['response'],
    ]
}

# トラジェクトリ評価用のデータ
trajectory_eval_data = {
    'prompt': [
        query1,
        query2,
    ],
    'reference_trajectory': [
        events1['predicted_trajectory'],
        events2['predicted_trajectory'],
    ],
}

## レスポンス評価

ここでは、Vertex AI Gen AI Evaluation Serviceが提供する、[ルーブリックベースの評価](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval?hl=ja)を行います。

- ルーブリックベースの評価では、**正解データを参照せずに**エージェントの出力結果の品質を評価します。
- 評価の観点は、ユーザーが指定するか、もしくは、GENERAL_QUALITY メトリックを用いて、評価の観点を自動生成します。
- ここでは、先に用意した「正解データ」に含まれる出力結果を GENERAL_QUALITY メトリックで評価します。出力ごとに評価の観点が自動で与えられます。

In [11]:
client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version='v1beta1', base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
    ),
)

# GENERAL_QUALITY レシピを使用してルーブリックを生成
data_with_rubrics = client.evals.generate_rubrics(
    src=pd.DataFrame(response_eval_data),
    rubric_group_name='general_quality_rubrics',
    predefined_spec_name=RubricMetric.GENERAL_QUALITY,
)

# 評価に使用するルーブリックグループを指定して実行
eval_result = client.evals.evaluate(
    dataset=data_with_rubrics,
    metrics=[RubricMetric.GENERAL_QUALITY(
      rubric_group_name='general_quality_rubrics',
    )],
)

Computing Metrics for Evaluation Dataset: 100%|██████████| 2/2 [00:08<00:00,  4.01s/it]


In [12]:
result = []
for case_index, case in enumerate(eval_result.eval_case_results):
    result.append(f'\n# Query {case_index+1}')
    result.append(response_eval_data['prompt'][case_index])
    result.append('## Response')
    result.append(response_eval_data['response'][case_index])
    for item in case.response_candidate_results:
        result.append('## Evaluation')
        for rubric_index, rubric in enumerate(item.metric_results['general_quality_v1'].rubric_verdicts):
            result.append(f'### Rubric: {rubric.evaluated_rubric.content.property.description}')
            result.append(f'- Importance: {str(rubric.evaluated_rubric.importance).split(".")[-1]}')
            result.append(f'- Satisfied: {bool(rubric.verdict)}')
            result.append(f'- Reason: {rubric.reasoning}')

display(Markdown('\n'.join(result)))


# Query 1
健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。
## Response
健保のゴールドプランの個人の免責額は5万円です。
## Evaluation
### Rubric: The response is in Japanese.
- Importance: HIGH
- Satisfied: True
- Reason: The entire response is written in the Japanese language.
### Rubric: The response clarifies the terminology '健保のゴールドプラン' (Kenpo no Gold Plan) in the context of Japanese health insurance.
- Importance: HIGH
- Satisfied: False
- Reason: The response does not clarify the terminology '健保のゴールドプラン'. Instead, it directly provides a deductible amount for it, implying its existence without any explanation or clarification about its context within Japanese health insurance.
### Rubric: The response explains that Japanese public health insurance (健保) typically does not use a 'Gold Plan' tier or a '免責額' (deductible) in the same way some private or foreign insurance systems might.
- Importance: HIGH
- Satisfied: False
- Reason: The response directly provides a deductible amount for a '健保のゴールドプラン', which implies the existence and use of such a plan and deductible within Japanese public health insurance, rather than explaining that these terms are not typically used.
### Rubric: The response clarifies that Japanese public health insurance uses co-payment rates (自己負担割合) instead of deductibles.
- Importance: HIGH
- Satisfied: False
- Reason: The response does not mention co-payment rates (自己負担割合) at all, nor does it clarify that they are used instead of deductibles in Japanese public health insurance.
### Rubric: The response provides general information on typical co-payment rates for Japanese public health insurance.
- Importance: MEDIUM
- Satisfied: False
- Reason: The response does not provide any information regarding typical co-payment rates for Japanese public health insurance.
### Rubric: The response advises the user on how to find specific deductible information if they are referring to a private health insurance plan.
- Importance: MEDIUM
- Satisfied: False
- Reason: The response provides a direct answer for a specific plan and does not offer any advice on how to find information for private health insurance plans.
### Rubric: The response acknowledges the user's stated purpose of planning for an upcoming surgery.
- Importance: LOW
- Satisfied: False
- Reason: The response directly answers the question about the deductible but does not acknowledge the user's stated purpose of planning for an upcoming surgery.
### Rubric: The response recommends consulting with their specific insurance provider or the hospital for accurate cost estimates for their surgery.
- Importance: MEDIUM
- Satisfied: False
- Reason: The response provides a direct answer to the question and does not include any recommendations to consult with an insurance provider or the hospital for accurate cost estimates.

# Query 2
金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。
## Response
金曜日はカジュアルな服装が認められていますが、お客様との打ち合わせがある場合は、ビジネスカジュアルが適切です。
## Evaluation
### Rubric: The response is in Japanese.
- Importance: HIGH
- Satisfied: True
- Reason: The entire response is written in the Japanese language.
### Rubric: The response states the dress code for Friday.
- Importance: HIGH
- Satisfied: True
- Reason: The response explicitly mentions "金曜日" (Friday) and then describes the dress code for that day.
### Rubric: The response describes the nature or type of the dress code (e.g., business formal, business casual, casual, etc.).
- Importance: MEDIUM
- Satisfied: True
- Reason: The response uses the terms "カジュアルな服装" (casual attire) and "ビジネスカジュアル" (business casual) to describe the dress code.
### Rubric: The response acknowledges the user's purpose of informing a client about the dress code.
- Importance: LOW
- Satisfied: True
- Reason: The response directly addresses the user's mention of a client meeting ("お客様との打ち合わせ") and provides a dress code recommendation specifically for that situation, showing it understood the user's purpose.

## トラジェクトリ評価

ここでは、Vertex AI Gen AI Evaluation Serviceが提供する、[軌跡評価](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/evaluation-agents?hl=ja)を行います。

- トラジェクトリ評価では、エージェントによるツールの使用順序が**事前に用意した正解データと一致しているか**を評価します。
- 正解データに含まれるクエリを順番に再実行して、それぞれの結果を正解データの結果と比較します。
- このノートブックでは、正解データに複数のクエリがある場合、同一のセッションを用いてクエリを実行するように実装しています。

### シングルメトリック評価

シングルメトリック評価は、**エージェントが特定のツールを使用したかどうか**をチェックします。

- ここでは一例として、`log_policy_audit` を使用したかどうかをチェックします。
- 正解データと同じインストラクションのまま、temperature を 0.4 に上げてテストします。

In [13]:
tool_name = 'log_policy_audit' # チェック対象のツール名

EXPERIMENT_RUN = f"single-metric-eval-{get_id()}"

single_tool_usage_metrics = [TrajectorySingleToolUse(tool_name=tool_name)]

single_tool_call_eval_task = EvalTask(
    dataset=trajectory_eval_data,
    metrics=single_tool_usage_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/single-metric-eval",
)

# セッションをリセット
chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.4))
single_tool_call_eval_result = single_tool_call_eval_task.evaluate(
    runnable=chat_client.agent_parsed_outcome_sync,
    experiment_run_name=EXPERIMENT_RUN
)

Associating projects/879055303739/locations/us-central1/metadataStores/default/contexts/evaluate-adk-agent-single-metric-eval-c8c010b2 to Experiment: evaluate-adk-agent


Logging Eval experiment evaluation metadata: {'output_file': 'gs://etsuji-15pro-poc/single-metric-eval/eval_results_2026-01-09-06-48-01-178a4.csv'}


100%|██████████| 2/2 [00:09<00:00,  4.84s/it]

All 2 responses are successfully generated from the runnable.
Computing metrics with a total of 2 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 2/2 [00:00<00:00, 15.37it/s]

All 2 metric requests are successfully computed.
Evaluation Took:0.14271124400329427 seconds


In [14]:
print(json.dumps(single_tool_call_eval_result.summary_metrics, indent=2))

{
  "row_count": 2,
  "trajectory_single_tool_use/mean": 1.0,
  "trajectory_single_tool_use/std": 0.0,
  "latency_in_seconds/mean": 9.392629111998758,
  "latency_in_seconds/std": 0.41251908588616215,
  "failure/mean": 0.0,
  "failure/std": 0.0
}


In [15]:
for _, item in single_tool_call_eval_result.metrics_table[
    ['prompt', 'trajectory_single_tool_use/score']
].iterrows():
    print(f'Query: {item.prompt}')
    print(f'Success: {bool(item["trajectory_single_tool_use/score"])}')

Query: 健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。
Success: True
Query: 金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。
Success: True


いずれのクエリーでも `log_policy_audit` が使用されました。

### トラジェクトリ評価

トラジェクトリ評価は、エージェントによるツールの使用順序を正解データと比較します。

厳密度の高い順に、次の3つの比較方法があります。

- `trajectory_exact_match`: ツールの使用順序が正解データと完全に一致しているかチェックします。
- `trajectory_in_order_match`: ツールの使用順序が正解データと一致しているかチェックします。正解データに含まれないツール使用は無視します。
- `trajectory_any_order_match`: ツールの使用順序に関係なく、正解データに含まれるツール使用がすべて含まれているかチェックします。

また、ツール使用の正確性と網羅性を表す次の指標も計算されます。

- `trajectory_precision`: ツール使用の正確性を 0〜1 の値で示します。正解データに含まれないツール使用があると値が小さくなります。
- `trajectory_recall`: ツール使用の網羅性を 0〜1 の値で示します。正解データに含まれるツール使用が欠けていると値が小さくなります。

まず、正解データと同じインストラクションのまま、temperature を 0.4 に上げてテストします。

In [16]:
trajectory_metrics = [
    "trajectory_exact_match",
    "trajectory_in_order_match",
    "trajectory_any_order_match",
    "trajectory_precision",
    "trajectory_recall",
]

EXPERIMENT_RUN = f"trajectory-{get_id()}"

trajectory_eval_task = EvalTask(
    dataset=trajectory_eval_data,
    metrics=trajectory_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/multiple-metric-eval",
)

chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.4))
trajectory_eval_result = trajectory_eval_task.evaluate(
    runnable=chat_client.agent_parsed_outcome_sync,
    experiment_run_name=EXPERIMENT_RUN
)

Associating projects/879055303739/locations/us-central1/metadataStores/default/contexts/evaluate-adk-agent-trajectory-59879d34 to Experiment: evaluate-adk-agent


Logging Eval experiment evaluation metadata: {'output_file': 'gs://etsuji-15pro-poc/multiple-metric-eval/eval_results_2026-01-09-06-48-13-e42f9.csv'}


100%|██████████| 2/2 [00:08<00:00,  4.17s/it]

All 2 responses are successfully generated from the runnable.
Computing metrics with a total of 10 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 10/10 [00:00<00:00, 10.20it/s]

All 10 metric requests are successfully computed.
Evaluation Took:0.9951788670005044 seconds


In [17]:
print(json.dumps(trajectory_eval_result.summary_metrics, indent=2))

{
  "row_count": 2,
  "trajectory_exact_match/mean": 0.5,
  "trajectory_exact_match/std": 0.7071067811865476,
  "trajectory_in_order_match/mean": 0.5,
  "trajectory_in_order_match/std": 0.7071067811865476,
  "trajectory_any_order_match/mean": 0.5,
  "trajectory_any_order_match/std": 0.7071067811865476,
  "trajectory_precision/mean": 0.83333335,
  "trajectory_precision/std": 0.23570223682528985,
  "trajectory_recall/mean": 0.83333335,
  "trajectory_recall/std": 0.23570223682528985,
  "latency_in_seconds/mean": 7.913670305000778,
  "latency_in_seconds/std": 0.5849361793784926,
  "failure/mean": 0.0,
  "failure/std": 0.0
}


In [18]:
trajectory_eval_result.metrics_table[[
    'trajectory_exact_match/score',
    'trajectory_in_order_match/score',
    'trajectory_any_order_match/score',
]]

,trajectory_exact_match/score,trajectory_in_order_match/score,trajectory_any_order_match/score
0,0.0,0.0,0.0
1,1.0,1.0,1.0


In [19]:
trajectory_eval_result.metrics_table[[
    'trajectory_precision/score',
    'trajectory_recall/score',
]]

,trajectory_precision/score,trajectory_recall/score
0,0.666667,0.666667
1,1.000000,1.000000


- 1つめのクエリーは `trajectory_any_order_match` が 0 なので、正解データに含まれるツール使用の中に、今回は実行されなかったものが存在します。
- 2つめのクエリーは `trajectory_exact_match` が 1.0 なので、ツールの使用は正解データと完全に一致しています。

正解データのトラジェクトリ（Reference Trajectory）と再実行時のトラジェクトリ（Predicted Trajectory）を表示して確認します。

- 1つめのクエリーでは、`log_policy_audit` に与える `reason` オプションが `customer` に変わっているため、正解データのツール使用とは一致しないものとみなされました。

In [20]:
for c, item in trajectory_eval_result.metrics_table.iterrows():
    print('\n== Query ==')
    print(item.prompt)
    print('\n== Reference Trajectory ==')
    print(json.dumps(item.reference_trajectory, indent=2, sort_keys=True))
    print('\n== Predicted Trajectory ==')
    print(json.dumps(item.predicted_trajectory, indent=2, sort_keys=True))


== Query ==
健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。

== Reference Trajectory ==
[
  {
    "tool_input": {
      "category": "benefits"
    },
    "tool_name": "list_policy_documents"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold"
    },
    "tool_name": "get_document_text"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold",
      "reason": "personal"
    },
    "tool_name": "log_policy_audit"
  }
]

== Predicted Trajectory ==
[
  {
    "tool_input": {
      "category": "benefits"
    },
    "tool_name": "list_policy_documents"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold"
    },
    "tool_name": "get_document_text"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold",
      "reason": "customer"
    },
    "tool_name": "log_policy_audit"
  }
]

== Query ==
金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。

== Reference Trajectory ==
[
  {
    "tool_input": {
      "category": "hr"
    },
    "tool_name": "list_policy_documents"
  },
  {
    "tool

次は、インストラクションを変更して、必ず、複数のドキュメントを参照するように指示します。

In [23]:
instruction='''
あなたは人事アシスタントです。
1. list_policy_documents を使用してドキュメントIDを検索してください。
2. get_document_text を使用して内容を取得してください。
3. ドキュメントのテキストに基づいてユーザーの質問に答えてください。

markdownを使用せずに、最終回答のみを日本語で出力します。

**重要**
- 確認漏れを防ぐために、必ず2種類以上のドキュメントを参照して回答すること。
- ドキュメントを読んだ後は、監査のために必ず log_policy_audit を使用してアクセスを記録すること。
'''

EXPERIMENT_RUN = f"trajectory-{get_id()}"

trajectory_eval_task = EvalTask(
    dataset=trajectory_eval_data,
    metrics=trajectory_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/multiple-metric-eval",
)

chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.0))
trajectory_eval_result = trajectory_eval_task.evaluate(
    runnable=chat_client.agent_parsed_outcome_sync,
    experiment_run_name=EXPERIMENT_RUN
)

Associating projects/879055303739/locations/us-central1/metadataStores/default/contexts/evaluate-adk-agent-trajectory-f8a86c36 to Experiment: evaluate-adk-agent


Logging Eval experiment evaluation metadata: {'output_file': 'gs://etsuji-15pro-poc/multiple-metric-eval/eval_results_2026-01-09-06-55-00-b4cb0.csv'}


100%|██████████| 2/2 [00:22<00:00, 11.25s/it]

All 2 responses are successfully generated from the runnable.
Computing metrics with a total of 10 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 10/10 [00:00<00:00, 10.01it/s]

All 10 metric requests are successfully computed.
Evaluation Took:1.0125558430008823 seconds


In [24]:
trajectory_eval_result.metrics_table[[
    'trajectory_exact_match/score',
    'trajectory_in_order_match/score',
    'trajectory_any_order_match/score',
]]

,trajectory_exact_match/score,trajectory_in_order_match/score,trajectory_any_order_match/score
0,0.0,1.0,1.0
1,0.0,1.0,1.0


In [25]:
trajectory_eval_result.metrics_table[[
    'trajectory_precision/score',
    'trajectory_recall/score',
]]

,trajectory_precision/score,trajectory_recall/score
0,0.60,1.0
1,0.75,1.0


- いずれのクエリーも `trajectory_exact_match` が 0 なので、ツールの使用は正解データと完全には一致していません。
- いずれのクエリーも `trajectory_in_order_match` が 1.0 なので、正解データには含まれないツール使用が追加されたことがわかります。
- その結果 `trajectory_precision` の値が下がっています。

正解データのトラジェクトリ（Reference Trajectory）と再実行時のトラジェクトリ（Predicted Trajectory）を表示して確認します。

In [26]:
for c, item in trajectory_eval_result.metrics_table.iterrows():
    print('\n== Query ==')
    print(item.prompt)
    print('\n== Reference Trajectory ==')
    print(json.dumps(item.reference_trajectory, indent=2, sort_keys=True))
    print('\n== Predicted Trajectory ==')
    print(json.dumps(item.predicted_trajectory, indent=2, sort_keys=True))


== Query ==
健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。

== Reference Trajectory ==
[
  {
    "tool_input": {
      "category": "benefits"
    },
    "tool_name": "list_policy_documents"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold"
    },
    "tool_name": "get_document_text"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold",
      "reason": "personal"
    },
    "tool_name": "log_policy_audit"
  }
]

== Predicted Trajectory ==
[
  {
    "tool_input": {
      "category": "benefits"
    },
    "tool_name": "list_policy_documents"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold"
    },
    "tool_name": "get_document_text"
  },
  {
    "tool_input": {
      "doc_id": "ben_401k_match"
    },
    "tool_name": "get_document_text"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold",
      "reason": "personal"
    },
    "tool_name": "log_policy_audit"
  },
  {
    "tool_input": {
      "doc_id": "ben_401k_match",
      "reason": "personal"
    },
   

インストラクションを変更して、`log_policy_audit` を使用するタイミングを変更します。

In [27]:
instruction='''
あなたは人事アシスタントです。
1. list_policy_documents を使用してドキュメントIDを検索してください。
2. get_document_text を使用して内容を取得してください。
3. ドキュメントのテキストに基づいてユーザーの質問に答えてください。

markdownを使用せずに、最終回答のみを日本語で出力します。

**重要**
- ドキュメントを読む前に、監査のために必ず log_policy_audit を使用してアクセスを記録すること。
'''

EXPERIMENT_RUN = f"trajectory-{get_id()}"

trajectory_eval_task = EvalTask(
    dataset=trajectory_eval_data,
    metrics=trajectory_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/multiple-metric-eval",
)

chat_client = ChatClient(crate_adkapp(instruction=instruction, temperature=0.0))
trajectory_eval_result = trajectory_eval_task.evaluate(
    runnable=chat_client.agent_parsed_outcome_sync,
    experiment_run_name=EXPERIMENT_RUN
)

Associating projects/879055303739/locations/us-central1/metadataStores/default/contexts/evaluate-adk-agent-trajectory-874ae1a5 to Experiment: evaluate-adk-agent


Logging Eval experiment evaluation metadata: {'output_file': 'gs://etsuji-15pro-poc/multiple-metric-eval/eval_results_2026-01-09-06-55-25-c1033.csv'}


100%|██████████| 2/2 [00:08<00:00,  4.13s/it]

All 2 responses are successfully generated from the runnable.
Computing metrics with a total of 10 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 10/10 [00:00<00:00, 10.70it/s]

All 10 metric requests are successfully computed.
Evaluation Took:0.9492438289998972 seconds


In [28]:
trajectory_eval_result.metrics_table[[
    'trajectory_exact_match/score',
    'trajectory_in_order_match/score',
    'trajectory_any_order_match/score',
]]

,trajectory_exact_match/score,trajectory_in_order_match/score,trajectory_any_order_match/score
0,0.0,0.0,1.0
1,0.0,0.0,1.0


In [29]:
trajectory_eval_result.metrics_table[[
    'trajectory_precision/score',
    'trajectory_recall/score',
]]

,trajectory_precision/score,trajectory_recall/score
0,1.0,1.0
1,1.0,1.0


- いずれのクエリーも `trajectory_in_order_match` が 0 なので、正解データとはツール使用の順序が異なることがわかります。
- いずれのクエリーも `trajectory_any_order_match` が 1.0 なので、使用順序を無視すれば、正解データに含まれるツール使用はすべて含まれることがわかります。
- `trajectory_precision` が 1.0 なので、正解データに含まれないツール使用はなかったことがわかります。

正解データのトラジェクトリ（Reference Trajectory）と再実行時のトラジェクトリ（Predicted Trajectory）を表示して確認します。

In [30]:
for c, item in trajectory_eval_result.metrics_table.iterrows():
    print('\n== Query ==')
    print(item.prompt)
    print('\n== Reference Trajectory ==')
    print(json.dumps(item.reference_trajectory, indent=2, sort_keys=True))
    print('\n== Predicted Trajectory ==')
    print(json.dumps(item.predicted_trajectory, indent=2, sort_keys=True))


== Query ==
健保のゴールドプランの免責額を知る必要があります。今度の手術の計画を立てるためです。

== Reference Trajectory ==
[
  {
    "tool_input": {
      "category": "benefits"
    },
    "tool_name": "list_policy_documents"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold"
    },
    "tool_name": "get_document_text"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold",
      "reason": "personal"
    },
    "tool_name": "log_policy_audit"
  }
]

== Predicted Trajectory ==
[
  {
    "tool_input": {
      "category": "benefits"
    },
    "tool_name": "list_policy_documents"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold",
      "reason": "personal"
    },
    "tool_name": "log_policy_audit"
  },
  {
    "tool_input": {
      "doc_id": "ben_med_gold"
    },
    "tool_name": "get_document_text"
  }
]

== Query ==
金曜日のドレスコードは何ですか？オフィスで打ち合わせがあるのでお客様に伝えておきます。

== Reference Trajectory ==
[
  {
    "tool_input": {
      "category": "hr"
    },
    "tool_name": "list_policy_documents"
  },
  {
    "tool